# Symmetric Matrix Vector Multiply Routines (stored in lower triangle)

This notebook walks you through how to implement $ y := A x + y $ where $ A $ is symmetric with only the lower triangular part stored.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with the "import laff as laff" and "import flame" statements.

## Algorithm that takes dot products

## The routine <br> <code> Symv_l_unb_var1( A, x, y ) </code>

This routine, given symmetric matrix $ A \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := A x + y $.  The "_l_" in the name of the routine indicates that $ A $ is stored in the lower triangular part of the matrix.

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [1]:
import flame
import laff as laff

def Symv_l_unb_var1(A, x, y):

    ATL, ATR, \
    ABL, ABR  = flame.part_2x2(A, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    yT, \
    yB  = flame.part_2x1(y, \
                         0, 'TOP')

    while ATL.shape[0] < A.shape[0]:

        A00,  a01,     A02,  \
        a10t, alpha11, a12t, \
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, \
                                                       ABL, ABR, \
                                                       1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        y0,   \
        psi1, \
        y2    = flame.repart_2x1_to_3x1(yT, \
                                        yB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.dots(a10t, x0, psi1)
        laff.dots(alpha11, chi1, psi1)
        laff.dots(a21, x2, psi1)

        #------------------------------------------------------------#

        ATL, ATR, \
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  \
                                               a10t, alpha11, a12t, \
                                               A20,  a21,     A22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

        yT, \
        yB  = flame.cont_with_3x1_to_2x1(y0,   \
                                         psi1, \
                                         y2,   \
                                         'TOP')

    flame.merge_2x1(yT, \
                    yB, y)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [2]:
from numpy import random
from numpy import matrix

A = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
y = matrix( random.rand( 4,1 ) )
yold = matrix( random.rand( 4,1 ) )

print( 'A before =' )
print( A )

print( 'x before =' )
print( x )

print( 'y before =' )
print( y )

A before =
[[0.46832097 0.32560235 0.1431817  0.31886845]
 [0.90391453 0.17928059 0.3192057  0.18893547]
 [0.35966    0.34334084 0.68387113 0.4769394 ]
 [0.10944408 0.70015566 0.00099677 0.10487572]]
x before =
[[0.8680642 ]
 [0.01136214]
 [0.9288396 ]
 [0.08552156]]
y before =
[[0.56005456]
 [0.0352329 ]
 [0.05962158]
 [0.43873417]]


In [3]:
import numpy as np

# Notice that A is not symmetric.  We now "symmetrize it"
Asymm = np.tril( A ) + np.transpose( np.tril( A, -1 ) )

laff.copy( y, yold )   # save the original vector y

Symv_l_unb_var1( A, x, y )

print( 'y after =' )
print( y )

print( 'y - ( Asymm * x + yold ) = ' )
print( y - ( Asymm * x + yold ) )

y after =
[[1.3202839 ]
 [1.20071273]
 [1.01102246]
 [0.5515889 ]]
y - ( Asymm * x + yold ) = 
[[2.22044605e-16]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

## The routine <br> <code> Symv_l_unb_var2( A, x, y ) </code>

This routine, given symmetric matrix $ A \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := A x + y $.  The "_l_" in the name of the routine indicates that $ A $ is stored in the lower triangular part of the matrix.

The specific laff functions we will use are
<ul>
<li> <code> laff.axpy( alpha, x, y ) </code> which computes $ y := \alpha x + y $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [4]:
def Symv_l_unb_var2(A, x, y):

    ATL, ATR, \
    ABL, ABR  = flame.part_2x2(A, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    yT, \
    yB  = flame.part_2x1(y, \
                         0, 'TOP')

    while ATL.shape[0] < A.shape[0]:

        A00,  a01,     A02,  \
        a10t, alpha11, a12t, \
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, \
                                                       ABL, ABR, \
                                                       1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        y0,   \
        psi1, \
        y2    = flame.repart_2x1_to_3x1(yT, \
                                        yB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.axpy(chi1, a10t, y0)
        laff.axpy(chi1, alpha11, psi1)
        laff.axpy(chi1, a21, y2)

        #------------------------------------------------------------#

        ATL, ATR, \
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  \
                                               a10t, alpha11, a12t, \
                                               A20,  a21,     A22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

        yT, \
        yB  = flame.cont_with_3x1_to_2x1(y0,   \
                                         psi1, \
                                         y2,   \
                                         'TOP')

    flame.merge_2x1(yT, \
                    yB, y)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [5]:
from numpy import random
from numpy import matrix

A = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
y = matrix( random.rand( 4,1 ) )
yold = matrix( random.rand( 4,1 ) )

print( 'A before =' )
print( A )

print( 'x before =' )
print( x )

print( 'y before =' )
print( y )

A before =
[[0.15873719 0.08931456 0.26411798 0.44761467]
 [0.26156822 0.54085484 0.49002409 0.59354755]
 [0.29539232 0.57437061 0.42329874 0.71895535]
 [0.27537768 0.06461721 0.11673445 0.80410785]]
x before =
[[0.53728628]
 [0.60169273]
 [0.52253892]
 [0.8432837 ]]
y before =
[[0.07148324]
 [0.27898294]
 [0.58972358]
 [0.62860295]]


In [6]:
import numpy as np

# Notice that A is not symmetric.  We now "symmetrize it"
Asymm = np.tril( A ) + np.transpose( np.tril( A, -1 ) )

laff.copy( y, yold )   # save the original vector y

Symv_l_unb_var2( A, x, y )

print( 'y after =' )
print( y )

print( 'y - ( Asymm * x + yold ) = ' )
print( y - ( Asymm * x + yold ) )

y after =
[[0.70072974]
 [1.09957002]
 [1.41365878]
 [1.55452864]]
y - ( Asymm * x + yold ) = 
[[0.00000000e+00]
 [0.00000000e+00]
 [2.22044605e-16]
 [0.00000000e+00]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.